In [ ]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input, decode_predictions
import numpy as np
import random
import math
import os
import json
from keras.models import load_model

vgg_path_str_old = 'vgg_19_0519'
vgg_path_str_new = 'vgg_19_10'
# model_vgg19 = VGG19(weights='imagenet')
# model_vgg19.summary()

save_dir = os.path.join(os.getcwd(), 'vgg19_saved_models')
model_path = os.path.join(save_dir, vgg_path_str_old+'.h5')
model_vgg19 = load_model(model_path)
total_layer = model_vgg19.layers
layer_index_size = 19
original_size = 143667240

N_bit = 16
fault_ratio = 0.005
random_bit_flip_position = [1,2,4,8,16,32,64,128,256,512,1024,2048,4096,8192,16384,32768]
global_gap = (2**N_bit)-1
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
for iteration in range(20):
    i = 0
    j = 0
    weights_store = []
    weights_store_float_to_int = np.zeros(original_size,dtype = int)
    weights_store_int_to_float = []
    layer_weight_size = np.zeros(layer_index_size*2,dtype = int)
    for layer_index in range(np.size(total_layer)):
        layer_weight_load = model_vgg19.layers[layer_index].get_weights() 
        if(len(layer_weight_load) > 0):
#             print('aaaaaaaaaaaaaaaaaaa')
            model_original_weights = layer_weight_load[0]
            model_original_bias = layer_weight_load[1]
            size_model_weights = np.size(model_original_weights)
            size_model_bias = np.size(model_original_bias)
            layer_weight_size[i] = size_model_weights
            i = i+1
            layer_weight_size[i] = size_model_bias
            i = i+1
            shape_model_weights = np.shape(model_original_weights)
            resized_model_weights = np.reshape(model_original_weights,(size_model_weights))
            weights_store_temp = np.append(resized_model_weights,model_original_bias)
            weights_store = np.append(weights_store,weights_store_temp)
    # print(layer_weight_size)
    # print(np.shape(weights_store))
    #####################################arrray reshape complete ######################################
    #####################################float convert to int and convert back start#####################################
    max_in_weights = np.max(weights_store)
    min_in_weights = np.min(weights_store)
    weights_store_float_to_int = np.around((weights_store-min_in_weights)/(max_in_weights-min_in_weights)*global_gap)
    weights_store_float_to_int = weights_store_float_to_int.astype(np.int)
    weights_store_int_max = np.max(weights_store_float_to_int)
    weights_store_int_min = np.min(weights_store_float_to_int)
    weights_store_int_to_float = weights_store_float_to_int/global_gap*(max_in_weights-min_in_weights)+min_in_weights
    weights_store_int_to_float = weights_store_int_to_float.astype(np.float32)
    difference_check = weights_store-weights_store_int_to_float
    difference_location = np.where(difference_check>(abs(max_in_weights-min_in_weights)/64))
    if np.size(difference_location)>0:
        print('conversion has error, certain value is out of threshold:',np.size(difference_location))
    ############################### random fault injection start ###################################
    fault_position_record = {}
    inject_index = [random.randint(0, np.size(weights_store_int_to_float)-1) for __ in range(math.ceil(np.size(weights_store_int_to_float)*fault_ratio))]
    for fault_index in inject_index:
        temp_weight_int_before_inject = np.around((weights_store_int_to_float[fault_index]-min_in_weights)/(max_in_weights-min_in_weights)*global_gap)
        temp_weight_int_before_inject = temp_weight_int_before_inject.astype(np.int)
    ############################## start inject ###############################################
        random_change_position = random.choice(random_bit_flip_position) #randomly choose one bit

    ############################ store index ###############################################
        inject_key = fault_index
        inject_value = [random_change_position]
        if not(inject_key in fault_position_record): # if no key add one
            fault_position_record[inject_key] = inject_value
            temp_weight_int_before_inject = temp_weight_int_before_inject ^ random_change_position
        else: #if has a key check the position
            while(random_change_position in fault_position_record[inject_key]):
                random_change_position = random.choice(random_bit_flip_position)
            fault_position_record[inject_key].append(random_change_position)
            temp_weight_int_before_inject = temp_weight_int_before_inject ^ random_change_position
    ############################## end inject #################################################
        temp_weight_int_after_inject = temp_weight_int_before_inject/global_gap*(max_in_weights-min_in_weights)+min_in_weights
        temp_weight_int_after_inject = temp_weight_int_after_inject.astype(np.float32)
    ###################################### fault injection end ###################################
        weights_store_int_to_float[fault_index] = temp_weight_int_after_inject
#     print('size of the fault position record:',len(fault_position_record))
    if sum([len(l) for l in fault_position_record.values()])- original_size*fault_ratio>1:
        print('compare:\n',sum([len(l) for l in fault_position_record.values()]),'and', original_size*fault_ratio)
    ############################### random fault injection end ###########################
    ##################################### accuracy check ############################################
    for layer in range(np.size(total_layer)):
        temp_weights_back = model_vgg19.layers[layer].get_weights()
        if(len(temp_weights_back) > 0):
#             print('length of the temp weights back:',len(temp_weights_back))
            model_weights_back = temp_weights_back[0]
            model_bias_back = temp_weights_back[1]
            shape_model_weights_back = np.shape(model_weights_back)
            size_model_weights_back = np.size(model_weights_back)
            resized_model_weights_back = np.reshape(model_weights_back,(size_model_weights_back))
            resized_model_weights_back = weights_store_int_to_float[0:(layer_weight_size[j])]
            weights_store_int_to_float = np.delete(weights_store_int_to_float,np.arange(layer_weight_size[j]))
            j = j+1
            model_bias_back = weights_store_int_to_float[0:(layer_weight_size[j])]
            weights_store_int_to_float = np.delete(weights_store_int_to_float,np.arange(layer_weight_size[j]))
            reshaped_model_weights_back = np.reshape(resized_model_weights_back,(shape_model_weights_back))
            temp_weights_back[0] = reshaped_model_weights_back
            temp_weights_back[1] = model_bias_back
            model_vgg19.layers[layer].set_weights(temp_weights_back)
            j = j+1
#             print('current size of the total weights store int to float:',np.size(weights_store_int_to_float))
    if np.size(weights_store_int_to_float)!=0:
        print('weights write back dimension mismatch:',np.size(weights_store_int_to_float))